## Imports 

In [165]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import iqr
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import pickle
import json

In [79]:
data = pd.read_csv('./survey_results_public.csv')

## Data exploration, Cleaning & Preprocessing

In [82]:
data.shape

(89184, 84)

In [84]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89184 entries, 0 to 89183
Data columns (total 84 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ResponseId                           89184 non-null  int64  
 1   Q120                                 89184 non-null  object 
 2   MainBranch                           89184 non-null  object 
 3   Age                                  89184 non-null  object 
 4   Employment                           87898 non-null  object 
 5   RemoteWork                           73810 non-null  object 
 6   CodingActivities                     73764 non-null  object 
 7   EdLevel                              87973 non-null  object 
 8   LearnCode                            87663 non-null  object 
 9   LearnCodeOnline                      70084 non-null  object 
 10  LearnCodeCoursesCert                 37076 non-null  object 
 11  YearsCode                   

In [85]:
data = data.drop(['ResponseId', 'Q120'], axis=1)

### taking only non CovertedCompYearly values

In [88]:
data = data[data["ConvertedCompYearly"].notnull()]

In [90]:
data.shape

(48019, 82)

### Turning to numeric values

In [94]:
data[['YearsCode', 'YearsCodePro']] = data[['YearsCode', 'YearsCodePro']].apply(pd.to_numeric, errors='coerce')

### Taking missing percentage threshold

In [97]:
missing_percentage_per_row = data.isnull().sum(axis=1) / data.shape[1]

# Count rows with more than 50% missing values
rows_with_high_missing = missing_percentage_per_row[missing_percentage_per_row >= 0.8]

# Print the count
print("Number of rows with more than 80% missing values:", len(rows_with_high_missing))

Number of rows with more than 80% missing values: 1


### Row wise

In [100]:
data = data[missing_percentage_per_row < 0.5]

In [102]:
data.shape

(45661, 82)

### Column wise

In [105]:
missing_values = data.isnull().mean() * 100
print(missing_values)

# Drop columns with more than a threshold (e.g., 80%) of missing values
threshold = 80
columns_to_drop = missing_values[missing_values > threshold].index
data = data.drop(columns_to_drop, axis=1)

MainBranch              0.000000
Age                     0.000000
Employment              0.021901
RemoteWork              0.113883
CodingActivities        0.155494
                         ...    
ProfessionalTech       30.533716
Industry               39.250126
SurveyLength            0.928582
SurveyEase              0.840980
ConvertedCompYearly     0.000000
Length: 82, dtype: float64


In [107]:
data.shape

(45661, 78)

### Selected columns cleaning

In [110]:
mandatory_columns1 = ['DevType', 'Industry', 'EdLevel', 'Age', 'Country', 'ProfessionalTech','WorkExp','YearsCodePro','LanguageHaveWorkedWith','RemoteWork','Employment','ToolsTechHaveWorkedWith','DatabaseHaveWorkedWith','WebframeHaveWorkedWith']

# Drop rows where any of the mandatory columns have null values
data = data.dropna(subset=mandatory_columns1)

In [112]:
print(data[mandatory_columns1].isnull().sum())

DevType                    0
Industry                   0
EdLevel                    0
Age                        0
Country                    0
ProfessionalTech           0
WorkExp                    0
YearsCodePro               0
LanguageHaveWorkedWith     0
RemoteWork                 0
Employment                 0
ToolsTechHaveWorkedWith    0
DatabaseHaveWorkedWith     0
WebframeHaveWorkedWith     0
dtype: int64


### Dropping columns

In [115]:
data.drop(columns=['YearsCode'], inplace=True)
data.drop(columns=['CompTotal'], inplace=True)
data.drop(columns=['Currency'], inplace=True)

### Outlier checking and removing

In [118]:
# List of columns to check for outliers
columns_to_check = ['WorkExp', 'YearsCodePro','ConvertedCompYearly']

# Create a copy of the original DataFrame to keep track of the original shape
original_shape = data.shape

# Initialize a mask for rows to drop with the same index as the DataFrame
mask_to_drop = pd.Series([False] * len(data), index=data.index)

# Loop through each specified column and identify outliers
for col in columns_to_check:
    # Calculate Q1 and Q3 for the current column
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    
    # Calculate the IQR for the current column
    IQR = Q3 - Q1
    
    # Update the mask to mark outliers for the current column
    mask_to_drop |= (data[col] < (Q1 - 1.5 * IQR)) | (data[col] > (Q3 + 1.5 * IQR))

# Drop the outliers from the original DataFrame
data = data[~mask_to_drop]

In [120]:
data.shape

(17728, 75)

### Numerical and categorical categorization

In [123]:
numerical_columns = data.select_dtypes(include=[float, int]).columns.tolist()
print("Numerical Columns:\n", numerical_columns)

# Identify categorical columns
categorical_columns = data.select_dtypes(include=['object']).columns
print("Categorical Columns:\n", categorical_columns)

Numerical Columns:
 ['YearsCodePro', 'WorkExp', 'ConvertedCompYearly']
Categorical Columns:
 Index(['MainBranch', 'Age', 'Employment', 'RemoteWork', 'CodingActivities',
       'EdLevel', 'LearnCode', 'LearnCodeOnline', 'LearnCodeCoursesCert',
       'DevType', 'OrgSize', 'PurchaseInfluence', 'TechList', 'BuyNewTool',
       'Country', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith',
       'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith',
       'PlatformHaveWorkedWith', 'PlatformWantToWorkWith',
       'WebframeHaveWorkedWith', 'WebframeWantToWorkWith',
       'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith',
       'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith',
       'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith',
       'OpSysPersonal use', 'OpSysProfessional use',
       'OfficeStackAsyncHaveWorkedWith', 'OfficeStackAsyncWantToWorkWith',
       'OfficeStackSyncHaveWorkedWith', 'OfficeStackSyncWantToWorkWith',
       'AISearchHaveWorkedWith', 'AISearch

### Corealtion matrix

In [126]:
target_column = 'ConvertedCompYearly'  

# Ensure target column is included in the numerical columns list
if target_column not in numerical_columns:
    numerical_columns.append(target_column)

# Calculate the correlation matrix
correlation_matrix = data[numerical_columns].corr()
print("Correlation Matrix:\n", correlation_matrix)

# Extract correlation with the target variable
target_corr = correlation_matrix[target_column].abs().sort_values(ascending=False)
print("Correlation with Target Variable:\n", target_corr)

# Set a threshold for correlation
threshold = 0.1 # Adjust this value as needed

# Select features with correlation higher than the threshold
important_numerical_features = target_corr[target_corr > threshold].index

# Drop the target column from the list of important features if needed
if target_column in important_numerical_features:
    important_numerical_features = important_numerical_features.drop(target_column)

print("Important Numerical Features:\n", important_numerical_features)

Correlation Matrix:
                      YearsCodePro   WorkExp  ConvertedCompYearly
YearsCodePro             1.000000  0.917913             0.360673
WorkExp                  0.917913  1.000000             0.355707
ConvertedCompYearly      0.360673  0.355707             1.000000
Correlation with Target Variable:
 ConvertedCompYearly    1.000000
YearsCodePro           0.360673
WorkExp                0.355707
Name: ConvertedCompYearly, dtype: float64
Important Numerical Features:
 Index(['YearsCodePro', 'WorkExp'], dtype='object')


### Content preprocessing

In [129]:
for col in categorical_columns:
    data[col] = data[col].str.strip().str.lower()

### Checking for null

In [132]:
for col in data :
  print(f'{col}\t\t{data[col].isna().sum()}')

MainBranch		0
Age		0
Employment		0
RemoteWork		0
CodingActivities		2
EdLevel		0
LearnCode		23
LearnCodeOnline		3041
LearnCodeCoursesCert		9172
YearsCodePro		0
DevType		0
OrgSize		6
PurchaseInfluence		3
TechList		5821
BuyNewTool		339
Country		0
LanguageHaveWorkedWith		0
LanguageWantToWorkWith		889
DatabaseHaveWorkedWith		0
DatabaseWantToWorkWith		2171
PlatformHaveWorkedWith		2250
PlatformWantToWorkWith		4602
WebframeHaveWorkedWith		0
WebframeWantToWorkWith		2048
MiscTechHaveWorkedWith		4793
MiscTechWantToWorkWith		6661
ToolsTechHaveWorkedWith		0
ToolsTechWantToWorkWith		1605
NEWCollabToolsHaveWorkedWith		36
NEWCollabToolsWantToWorkWith		1498
OpSysPersonal use		82
OpSysProfessional use		855
OfficeStackAsyncHaveWorkedWith		1256
OfficeStackAsyncWantToWorkWith		4292
OfficeStackSyncHaveWorkedWith		142
OfficeStackSyncWantToWorkWith		2449
AISearchHaveWorkedWith		6009
AISearchWantToWorkWith		7852
AIDevHaveWorkedWith		11460
AIDevWantToWorkWith		12834
NEWSOSites		0
SOVisitFreq		33
SOAccount		1
SO

In [134]:
def fill_categorical_target(df, column_name, target_column):
    # Calculate the mean of the target variable for each category
    means = df.groupby(column_name)[target_column].mean()
    
    # Fill null values in the categorical column with the mean of the target for that category
    df[column_name] = df[column_name].fillna(df[column_name].map(means))
    
    # Fallback: Fill remaining nulls with mode of the column
    mode_value = df[column_name].mode()[0]
    df[column_name].fillna(mode_value, inplace=True)

target_column = 'ConvertedCompYearly'  # Your target column

for col in categorical_columns:
    print(f"Processing categorical column: {col}")
    fill_categorical_target(data, col, target_column)

Processing categorical column: MainBranch
Processing categorical column: Age
Processing categorical column: Employment
Processing categorical column: RemoteWork
Processing categorical column: CodingActivities
Processing categorical column: EdLevel
Processing categorical column: LearnCode
Processing categorical column: LearnCodeOnline
Processing categorical column: LearnCodeCoursesCert
Processing categorical column: DevType


C:\Users\himak\AppData\Local\Temp\ipykernel_6544\1485541225.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].fillna(mode_value, inplace=True)
C:\Users\himak\AppData\Local\Temp\ipykernel_6544\1485541225.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

Processing categorical column: OrgSize
Processing categorical column: PurchaseInfluence
Processing categorical column: TechList
Processing categorical column: BuyNewTool
Processing categorical column: Country
Processing categorical column: LanguageHaveWorkedWith
Processing categorical column: LanguageWantToWorkWith
Processing categorical column: DatabaseHaveWorkedWith
Processing categorical column: DatabaseWantToWorkWith


C:\Users\himak\AppData\Local\Temp\ipykernel_6544\1485541225.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].fillna(mode_value, inplace=True)
C:\Users\himak\AppData\Local\Temp\ipykernel_6544\1485541225.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

Processing categorical column: PlatformHaveWorkedWith
Processing categorical column: PlatformWantToWorkWith
Processing categorical column: WebframeHaveWorkedWith
Processing categorical column: WebframeWantToWorkWith
Processing categorical column: MiscTechHaveWorkedWith
Processing categorical column: MiscTechWantToWorkWith
Processing categorical column: ToolsTechHaveWorkedWith


...

C:\Users\himak\AppData\Local\Temp\ipykernel_6544\1485541225.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].fillna(mode_value, inplace=True)
C:\Users\himak\AppData\Local\Temp\ipykernel_6544\1485541225.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

Processing categorical column: SOAI
Processing categorical column: AISelect
Processing categorical column: AISent
Processing categorical column: AIAcc
Processing categorical column: AIBen
Processing categorical column: AIToolInterested in Using
Processing categorical column: AIToolCurrently Using
Processing categorical column: AIToolNot interested in Using
Processing categorical column: AINextSomewhat different


C:\Users\himak\AppData\Local\Temp\ipykernel_6544\1485541225.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].fillna(mode_value, inplace=True)
C:\Users\himak\AppData\Local\Temp\ipykernel_6544\1485541225.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

Processing categorical column: TBranch
Processing categorical column: ICorPM
Processing categorical column: Knowledge_1
Processing categorical column: Knowledge_2
Processing categorical column: Knowledge_3
Processing categorical column: Knowledge_4
Processing categorical column: Knowledge_5
Processing categorical column: Knowledge_6
Processing categorical column: Knowledge_7
Processing categorical column: Knowledge_8
Processing categorical column: Frequency_1
Processing categorical column: Frequency_2
Processing categorical column: Frequency_3
Processing categorical column: TimeSearching
Processing categorical column: TimeAnswering
Processing categorical column: ProfessionalTech
Processing categorical column: Industry
Processing categorical column: SurveyLength
Processing categorical column: SurveyEase


C:\Users\himak\AppData\Local\Temp\ipykernel_6544\1485541225.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column_name].fillna(mode_value, inplace=True)
C:\Users\himak\AppData\Local\Temp\ipykernel_6544\1485541225.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [136]:
for col in data :
  print(f'{col}\t\t{data[col].isna().sum()}')

MainBranch		0
Age		0
Employment		0
RemoteWork		0
CodingActivities		0
EdLevel		0
LearnCode		0
LearnCodeOnline		0
LearnCodeCoursesCert		0
YearsCodePro		0
DevType		0
OrgSize		0
PurchaseInfluence		0
TechList		0
BuyNewTool		0
Country		0
LanguageHaveWorkedWith		0
LanguageWantToWorkWith		0
DatabaseHaveWorkedWith		0
DatabaseWantToWorkWith		0
PlatformHaveWorkedWith		0
PlatformWantToWorkWith		0
WebframeHaveWorkedWith		0
WebframeWantToWorkWith		0
MiscTechHaveWorkedWith		0
MiscTechWantToWorkWith		0
ToolsTechHaveWorkedWith		0
ToolsTechWantToWorkWith		0
NEWCollabToolsHaveWorkedWith		0
NEWCollabToolsWantToWorkWith		0
OpSysPersonal use		0
OpSysProfessional use		0
OfficeStackAsyncHaveWorkedWith		0
OfficeStackAsyncWantToWorkWith		0
OfficeStackSyncHaveWorkedWith		0
OfficeStackSyncWantToWorkWith		0
AISearchHaveWorkedWith		0
AISearchWantToWorkWith		0
AIDevHaveWorkedWith		0
AIDevWantToWorkWith		0
NEWSOSites		0
SOVisitFreq		0
SOAccount		0
SOPartFreq		0
SOComm		0
SOAI		0
AISelect		0
AISent		0
AIAcc		0
AIBen		

### Scaling numerical columns

In [128]:
data["YearsCodePro"].unique()

array([ 4., 21.,  3.,  9.,  7., 10.,  6., 14.,  2., 15., 28., 11.,  8.,
        5., 17., 16., 12., 20.,  1., 25., 22., 13., 23., 24., 27., 19.,
       26., 18.])

In [129]:
data["WorkExp"].unique()

array([ 6., 22.,  4.,  5.,  9.,  7., 15., 14., 18., 28., 12.,  3., 17.,
       10., 16., 11.,  2., 25.,  8., 20.,  1., 13., 30., 21., 23., 27.,
       26., 19., 24., 29.,  0.])

In [140]:
print(data["WorkExp"].min())
print(data["WorkExp"].max())
print(data["YearsCodePro"].min())
print(data["YearsCodePro"].max())

0.0
30.0
1.0
28.0


In [142]:
scaler = MinMaxScaler()
columns_to_scale = ['YearsCodePro', 'WorkExp']

# Apply the scaler only to the columns in columns_to_scale
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

### Selecting countries with certain value counts

In [145]:
def shorten_categories(categories, cutoff):
    categorical_map = {}
    for i in range(len(categories)):
        if categories.values[i] >= cutoff:
            categorical_map[categories.index[i]] = categories.index[i]
        else:
            categorical_map[categories.index[i]] = 'Other'
    return categorical_map

In [147]:
country_map = shorten_categories(data.Country.value_counts(), 400)
data['Country'] = data['Country'].map(country_map)

### Educationa column preprocessing

In [150]:
def clean_education(x):
    if 'Associate degree' in x:
        return 'Associate degree'
    if 'Bachelor’s degree' in x:
        return 'Bachelor’s degree'
    if 'Master’s degree' in x:
        return 'Master’s degree'
    if 'Professional degree' in x :
        return 'Post grad'   
    return 'Less than a Bachelors'

data["EdLevel"]=data["EdLevel"].apply(clean_education)

### Other column criterias

In [153]:
data=data[data['ConvertedCompYearly'] >= 10000]
data=data[data['Country'] != 'Other']

In [155]:
data = data[~data["DevType"].isin(["Other (please specify):", "Engineering manager","Academic researcher","Senior Executive (C-Suite, VP, etc.)","Security professional","Scientist","Product manager","Educator","Marketing or sales professional","Student"])]

In [157]:
data.shape

(10661, 75)

### 'Other' Grouping

In [160]:
# Define a threshold for rare categories
threshold = 20 # Categories with fewer than 20 occurrences will be grouped as 'Other'

# Iterate through each categorical column
for column in categorical_columns:
    # Get the frequency of each category
    value_counts = data[column].value_counts()
    
    # Identify categories that appear less than the threshold
    rare_categories = value_counts[value_counts < threshold].index
    
    # Replace rare categories with 'Other'
    data[column] = data[column].replace(rare_categories, 'Other')

print("Updated Data with Rare Categories Grouped as 'Other':\n", data.head())

Updated Data with Rare Categories Grouped as 'Other':
                         MainBranch              Age           Employment  \
5   i am a developer by profession  35-44 years old  employed, full-time   
6   i am a developer by profession  35-44 years old  employed, full-time   
7   i am a developer by profession  25-34 years old  employed, full-time   
12  i am a developer by profession  25-34 years old  employed, full-time   
14  i am a developer by profession  25-34 years old  employed, full-time   

                              RemoteWork  \
5                                 remote   
6                                 remote   
7                                 remote   
12                                remote   
14  hybrid (some remote, some in-person)   

                                     CodingActivities                EdLevel  \
5   hobby;professional development or self-paced l...  Less than a Bachelors   
6   hobby;contribute to open-source projects;profe...  Less tha

### Encoding

In [163]:
numerical_columns = data.select_dtypes(include=[float, int]).columns.tolist()
print("Numerical Columns:\n", numerical_columns)

# Identify categorical columns
categorical_columns = data.select_dtypes(include=['object']).columns
print("Categorical Columns:\n", categorical_columns)

Numerical Columns:
 ['YearsCodePro', 'WorkExp', 'ConvertedCompYearly']
Categorical Columns:
 Index(['MainBranch', 'Age', 'Employment', 'RemoteWork', 'CodingActivities',
       'EdLevel', 'LearnCode', 'LearnCodeOnline', 'LearnCodeCoursesCert',
       'DevType', 'OrgSize', 'PurchaseInfluence', 'TechList', 'BuyNewTool',
       'Country', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith',
       'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith',
       'PlatformHaveWorkedWith', 'PlatformWantToWorkWith',
       'WebframeHaveWorkedWith', 'WebframeWantToWorkWith',
       'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith',
       'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith',
       'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith',
       'OpSysPersonal use', 'OpSysProfessional use',
       'OfficeStackAsyncHaveWorkedWith', 'OfficeStackAsyncWantToWorkWith',
       'OfficeStackSyncHaveWorkedWith', 'OfficeStackSyncWantToWorkWith',
       'AISearchHaveWorkedWith', 'AISearch

### Categorical

In [167]:
label_encoder = LabelEncoder()

# Dictionary to store mappings for each categorical column
label_mappings = {}

for col in categorical_columns:
    data[col] = label_encoder.fit_transform(data[col])
    
    # Save the mapping (original category -> label)
    label_mappings[col] = {index: label for index, label in enumerate(label_encoder.classes_)}

# Print the saved mappings
(label_mappings)

{'MainBranch': {0: 'i am a developer by profession',
  1: 'i am not primarily a developer, but i write code sometimes as part of my work/studies'},
 'Age': {0: '18-24 years old',
  1: '25-34 years old',
  2: '35-44 years old',
  3: '45-54 years old',
  4: '55-64 years old',
  5: 'Other'},
 'Employment': {0: 'Other',
  1: 'employed, full-time',
  2: 'employed, full-time;employed, part-time',
  3: 'employed, full-time;independent contractor, freelancer, or self-employed',
  4: 'employed, part-time',
  5: 'independent contractor, freelancer, or self-employed',
  6: 'independent contractor, freelancer, or self-employed;employed, part-time'},
 'RemoteWork': {0: 'hybrid (some remote, some in-person)',
  1: 'in-person',
  2: 'remote'},
 'CodingActivities': {0: 'Other',
  1: 'bootstrapping a business',
  2: 'bootstrapping a business;freelance/contract work',
  3: 'bootstrapping a business;professional development or self-paced learning from online courses',
  4: 'bootstrapping a business;profe

In [169]:
# Save label_mappings to a JSON file
with open('label_mappings.json', 'w') as json_file:
    json.dump(label_mappings, json_file, indent=4)

print("Mappings saved to label_mappings.json")

Mappings saved to label_mappings.json


###  Selecting effective features

In [185]:
target_variable = 'ConvertedCompYearly'

# Compute the correlation matrix
correlation_matrix = data.corr()

# Extract correlation with the target variable
target_correlation = correlation_matrix[target_variable]

# Sorting
sorted_correlation = target_correlation.abs().sort_values(ascending=False)

#  Optionally set a threshold for strong correlation
threshold = 0.01
effective_features = sorted_correlation[sorted_correlation > threshold]

# Display the most effective features
print("Most Effective Features:\n", effective_features)

Most Effective Features:
 ConvertedCompYearly               1.000000
Country                           0.464049
YearsCodePro                      0.324244
WorkExp                           0.321126
Age                               0.274479
RemoteWork                        0.178003
OfficeStackSyncWantToWorkWith     0.116988
SOVisitFreq                       0.101137
Knowledge_1                       0.099860
Frequency_1                       0.096132
ToolsTechHaveWorkedWith           0.084363
OfficeStackSyncHaveWorkedWith     0.080172
WebframeWantToWorkWith            0.078342
Frequency_3                       0.073707
ICorPM                            0.073585
OpSysProfessional use             0.072779
LearnCode                         0.070464
LanguageHaveWorkedWith            0.069444
NEWSOSites                        0.065493
DevType                           0.059957
OfficeStackAsyncHaveWorkedWith    0.057257
CodingActivities                  0.055847
WebframeHaveWorkedWith      

In [187]:
high_effective_features = target_correlation[target_correlation.abs() > threshold].index.tolist()

# Step 2: List of mandatory columns that must always be included
mandatory_columns = ['DevType', 'Industry', 'EdLevel', 'Age', 'Country', 'ProfessionalTech', 
                     'WorkExp', 'YearsCodePro', 'LanguageHaveWorkedWith', 'RemoteWork', 
                     'Employment', 'ToolsTechHaveWorkedWith', 'DatabaseHaveWorkedWith', 
                     'WebframeHaveWorkedWith']

# Step 3: Ensure mandatory columns are in the final feature list
for col in mandatory_columns:
    if col not in high_effective_features and col in data.columns:  # Ensure the column exists in the dataset
        high_effective_features.append(col)

# Step 4: Create a new DataFrame with high-effective features and mandatory columns
new_data = data[high_effective_features]


# Step 5: Display the new DataFrame
print("New DataFrame with High-Effective Variables:\n", new_data)

New DataFrame with High-Effective Variables:
        Age  Employment  RemoteWork  CodingActivities  LearnCode  \
5        2           1           2                30         21   
6        2           1           2                23          0   
7        1           1           2                11         43   
12       1           1           2                30         81   
14       1           1           0                11         70   
...    ...         ...         ...               ...        ...   
89128    0           1           0                28         11   
89143    1           1           1                30         40   
89148    1           1           2                11         79   
89167    0           1           2                14         83   
89170    1           1           0                 2          0   

       LearnCodeOnline  YearsCodePro  DevType  OrgSize  PurchaseInfluence  \
5                    0      0.740741        8        3                  

## Train Test Split

In [190]:
data.shape

(10661, 75)

In [192]:
X = new_data.drop(columns=['ConvertedCompYearly'])  # Feature columns
y = new_data['ConvertedCompYearly']  # Target column (Salary ranges)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Model

In [196]:

param_dist = {
    'max_depth': [10,20,30],                # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],        # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],          # Minimum samples required at a leaf node
    'max_features': [None, 'auto', 'sqrt'], # Number of features to consider at each split
}

# Step 2: Initialize the DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state=42)

# Step 3: Set up RandomizedSearchCV for lightweight hyperparameter tuning
random_search = RandomizedSearchCV(estimator=dt_model, param_distributions=param_dist, 
                                   n_iter=3, cv=3, n_jobs=-1, random_state=42, verbose=1)

# Step 4: Fit the model with hyperparameter tuning
random_search.fit(X_train, y_train)

# Step 5: Get the best parameters and model
best_dt_model = random_search.best_estimator_
print(f"Best Parameters: {random_search.best_params_}")

# Step 6: Evaluate the model on training and test sets
train_r2 = best_dt_model.score(X_train, y_train)
test_r2 = best_dt_model.score(X_test, y_test)

print(f"Decision Tree model R² on training set: {train_r2:.2f}")
print(f"Decision Tree model R² on test set: {test_r2:.2f}")

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Best Parameters: {'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 10}
Decision Tree model R² on training set: 0.78
Decision Tree model R² on test set: 0.45


In [198]:
# Get feature importances
importances = best_dt_model.feature_importances_

# Create a DataFrame for feature importances
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

# Sort the DataFrame by importance and select the top 10 features
top_10_features = feature_importance_df.sort_values(by='Importance', ascending=False).head(5)
selected_features = top_10_features['Feature'].values.tolist()
# Display the top 10 important features
print("Top 10 important features:")
print(top_10_features)

# Select the top 10 features for model training
X_top_10 = X[top_10_features['Feature'].values]

Top 10 important features:
                  Feature  Importance
12                Country    0.608874
6            YearsCodePro    0.082063
46                WorkExp    0.068803
26  OpSysProfessional use    0.031889
60       ProfessionalTech    0.025916


In [200]:
for col in mandatory_columns:
    if col not in selected_features:
        selected_features.append(col)

newdata = X[selected_features].copy()

# If you also want to include the target variable 'y'
newdata['ConvertedCompYearly'] = y 

# Display the new DataFrame
print(newdata.head())

    Country  YearsCodePro   WorkExp  OpSysProfessional use  ProfessionalTech  \
5         9      0.740741  0.733333                     26                59   
6        10      0.074074  0.133333                     26                66   
7        10      0.074074  0.166667                     31                77   
12       10      0.296296  0.300000                      0                 8   
14        7      0.222222  0.233333                     26                 0   

    DevType  Industry  EdLevel  Age  LanguageHaveWorkedWith  RemoteWork  \
5         8         9        0    2                       0           2   
6        12         4        0    2                       0           2   
7        12         1        0    1                       0           2   
12       12         9        0    1                       0           2   
14       12         9        0    1                       0           0   

    Employment  ToolsTechHaveWorkedWith  DatabaseHaveWorkedWith  \
5

In [204]:
X_new = newdata.drop('ConvertedCompYearly', axis=1)  # If you included the target
y_new = newdata['ConvertedCompYearly']  # If you included the target


X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

### DecisionTree

In [208]:
# Step 0: Perform dimension reduction by removing highly correlated features
corr_matrix = X_train.corr()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than 0.6 (or another threshold)
to_drop = [column for column in upper.columns if any(upper[column] > 0.6)]

# Drop highly correlated features from the training and test sets
X_train_reduced = X_train.drop(to_drop, axis=1)
X_test_reduced = X_test.drop(to_drop, axis=1)

# Step 1: Define the hyperparameter distribution for DecisionTreeRegressor
param_dist = {
    'max_depth': [10, 20, 30],       # Max depth of the tree (None for no limit)
    'min_samples_split': [2, 5, 10], # Min samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],   # Min samples required at a leaf node
    'max_features': [None, 'auto', 'sqrt'] # Max features to consider for split
}

# Step 2: Initialize the DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state=42)

# Step 3: Set up RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(estimator=dt_model, 
                                   param_distributions=param_dist, 
                                   n_iter=10,            # Increased number of iterations for better tuning
                                   cv=3,                 # 3-fold cross-validation
                                   n_jobs=-1,            # Parallel processing
                                   random_state=42, 
                                   verbose=1)

# Step 4: Fit the model on the reduced training data with hyperparameter tuning
random_search.fit(X_train_reduced, y_train)

# Step 5: Extract the best model and hyperparameters
best_dt_model = random_search.best_estimator_
best_params = random_search.best_params_
print(f"Best Parameters: {best_params}")

# Step 6: Make predictions with the reduced dataset
y_train_pred_reduced = best_dt_model.predict(X_train_reduced)
y_test_pred_reduced = best_dt_model.predict(X_test_reduced)

# Step 7: Evaluate the model on the training and test sets with reduced data
train_r2_reduced = r2_score(y_train, y_train_pred_reduced)
train_mse_reduced = mean_squared_error(y_train, y_train_pred_reduced)
train_rmse_reduced = np.sqrt(train_mse_reduced)
train_mae_reduced = mean_absolute_error(y_train, y_train_pred_reduced)

test_r2_reduced = r2_score(y_test, y_test_pred_reduced)
test_mse_reduced = mean_squared_error(y_test, y_test_pred_reduced)
test_rmse_reduced = np.sqrt(test_mse_reduced)
test_mae_reduced = mean_absolute_error(y_test, y_test_pred_reduced)

# Step 8: Print the evaluation metrics
print(f"Decision Tree model R² on reduced training set: {train_r2_reduced:.2f}")
print(f"Decision Tree model R² on reduced test set: {test_r2_reduced:.2f}")
print(f"Decision Tree model MSE on reduced training set: {train_mse_reduced:.2f}")
print(f"Decision Tree model MSE on reduced test set: {test_mse_reduced:.2f}")
print(f"Decision Tree model RMSE on reduced training set: {train_rmse_reduced:.2f}")
print(f"Decision Tree model RMSE on reduced test set: {test_rmse_reduced:.2f}")
print(f"Decision Tree model MAE on reduced training set: {train_mae_reduced:.2f}")
print(f"Decision Tree model MAE on reduced test set: {test_mae_reduced:.2f}")


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Parameters: {'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': None, 'max_depth': 10}
Decision Tree model R² on reduced training set: 0.75
Decision Tree model R² on reduced test set: 0.55
Decision Tree model MSE on reduced training set: 679269622.96
Decision Tree model MSE on reduced test set: 1232104120.68
Decision Tree model RMSE on reduced training set: 26062.80
Decision Tree model RMSE on reduced test set: 35101.34
Decision Tree model MAE on reduced training set: 19261.13
Decision Tree model MAE on reduced test set: 25842.54


D:\Program Files\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
9 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
7 fits failed with the following error:
Traceback (most recent call last):
  File "D:\Program Files\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\Program Files\Anaconda\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "D:\Program Files\Anaconda\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "D:\Program Files\Anaconda\Lib\site-packag

### RandomForest

In [214]:

corr_matrix = X_train.corr()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than 0.6 (or another threshold)
to_drop = [column for column in upper.columns if any(upper[column] > 0.6)]

# Drop highly correlated features from the training and test sets
X_train_reduced = X_train.drop(to_drop, axis=1)
X_test_reduced = X_test.drop(to_drop, axis=1)

# Step 1: Define the hyperparameter distribution for RandomForestRegressor
param_dist = {
    'n_estimators': [100, 200, 300],        # Number of trees in the forest
    'max_depth': [10, 20, 30],              # Max depth of the tree (None for no limit)
    'min_samples_split': [2, 5, 10],        # Min samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],          # Min samples required at a leaf node
    'max_features': [None, 'auto', 'sqrt']  # Max features to consider for split
}

# Step 2: Initialize the RandomForestRegressor
rf_model = RandomForestRegressor(random_state=42)

# Step 3: Set up RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_model, 
                                   param_distributions=param_dist, 
                                   n_iter=10,            # Number of parameter settings sampled
                                   cv=3,                 # 3-fold cross-validation
                                   n_jobs=-1,            # Use all processors
                                   random_state=42, 
                                   verbose=1)

# Step 4: Fit the model on the reduced training data with hyperparameter tuning
random_search.fit(X_train_reduced, y_train)

# Step 5: Extract the best model and hyperparameters
best_rf_model = random_search.best_estimator_
best_params = random_search.best_params_
print(f"Best Parameters: {best_params}")

# Step 6: Evaluate the tuned model on the reduced training and test sets
train_r2 = best_rf_model.score(X_train_reduced, y_train)
test_r2 = best_rf_model.score(X_test_reduced, y_test)

# Predictions for reduced training and test sets
y_train_pred = best_rf_model.predict(X_train_reduced)
y_test_pred = best_rf_model.predict(X_test_reduced)

# Calculate MSE, RMSE, and MAE for reduced training set
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(y_train, y_train_pred)

# Calculate MSE, RMSE, and MAE for reduced test set
test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_test_pred)

# Print the results
print(f"Random Forest model R² on reduced training set: {train_r2:.2f}")
print(f"Random Forest model R² on reduced test set: {test_r2:.2f}")

print(f"Random Forest model MSE on reduced training set: {train_mse:.2f}")
print(f"Random Forest model MSE on reduced test set: {test_mse:.2f}")

print(f"Random Forest model RMSE on reduced training set: {train_rmse:.2f}")
print(f"Random Forest model RMSE on reduced test set: {test_rmse:.2f}")

print(f"Random Forest model MAE on reduced training set: {train_mae:.2f}")
print(f"Random Forest model MAE on reduced test set: {test_mae:.2f}")

# Step 7: Feature importance
importances = best_rf_model.feature_importances_
important_features = pd.Series(importances, index=X_train_reduced.columns).sort_values(ascending=False)

# Print the top 10 most important features
print("Top 10 Most Important Features:")
print(important_features.head(10))

print("Remaining columns in X_train after dropping highly correlated features:")
print(X_train_reduced.columns.tolist())

print("\nRemaining columns in X_test after dropping highly correlated features:")
print(X_test_reduced.columns.tolist())

Fitting 3 folds for each of 10 candidates, totalling 30 fits


D:\Program Files\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
6 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "D:\Program Files\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\Program Files\Anaconda\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "D:\Program Files\Anaconda\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "D:\Program Files\Anaconda\Lib\site-packag

Best Parameters: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 10}
Random Forest model R² on reduced training set: 0.75
Random Forest model R² on reduced test set: 0.65
Random Forest model MSE on reduced training set: 668776482.80
Random Forest model MSE on reduced test set: 966162040.58
Random Forest model RMSE on reduced training set: 25860.71
Random Forest model RMSE on reduced test set: 31083.15
Random Forest model MAE on reduced training set: 19226.23
Random Forest model MAE on reduced test set: 23130.31
Top 10 Most Important Features:
Country                   0.628363
YearsCodePro              0.167357
ProfessionalTech          0.054313
OpSysProfessional use     0.044313
DevType                   0.024635
WebframeHaveWorkedWith    0.018479
Industry                  0.017481
DatabaseHaveWorkedWith    0.016609
RemoteWork                0.008571
Employment                0.008119
dtype: float64
Remaining columns in X_train 

### GradientBoosting

In [219]:
# Step 1: Define the hyperparameter distribution for GradientBoostingRegressor
param_dist_gb = {
    'n_estimators': [100, 200, 300],         # Number of boosting stages
    'max_depth': [3, 5, 7],                  # Maximum depth of the individual trees
    'min_samples_split': [2, 5, 10],          # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],            # Minimum number of samples required at each leaf node
    'learning_rate': [0.01, 0.05, 0.1],      # Step size shrinkage to prevent overfitting
    'subsample': [0.8, 0.9, 1.0]              # The fraction of samples to be used for fitting the individual base learners
}

# Step 2: Initialize the GradientBoostingRegressor
gb_model = GradientBoostingRegressor(random_state=42)

# Step 3: Set up RandomizedSearchCV
random_search_gb = RandomizedSearchCV(estimator=gb_model, 
                                       param_distributions=param_dist_gb, 
                                       n_iter=10,               # Number of parameter settings sampled
                                       cv=3,                    # 3-fold cross-validation
                                       n_jobs=-1,               # Use all processors
                                       random_state=42, 
                                       verbose=1)

# Step 4: Fit the model on the training data with hyperparameter tuning
random_search_gb.fit(X_train, y_train)

# Step 5: Extract the best model and hyperparameters
best_gb_model = random_search_gb.best_estimator_
best_params_gb = random_search_gb.best_params_
print(f"Best Parameters for Gradient Boosting: {best_params_gb}")

# Step 6: Evaluate the tuned model on the training and test sets
train_r2_gb = best_gb_model.score(X_train, y_train)
test_r2_gb = best_gb_model.score(X_test, y_test)

# Predictions for training and test sets
y_train_pred_gb = best_gb_model.predict(X_train)
y_test_pred_gb = best_gb_model.predict(X_test)

# Calculate MSE, RMSE, and MAE for training set
train_mse_gb = mean_squared_error(y_train, y_train_pred_gb)
train_rmse_gb = np.sqrt(train_mse_gb)
train_mae_gb = mean_absolute_error(y_train, y_train_pred_gb)

# Calculate MSE, RMSE, and MAE for test set
test_mse_gb = mean_squared_error(y_test, y_test_pred_gb)
test_rmse_gb = np.sqrt(test_mse_gb)
test_mae_gb = mean_absolute_error(y_test, y_test_pred_gb)

# Print the results
print(f"Gradient Boosting model R² on training set: {train_r2_gb:.2f}")
print(f"Gradient Boosting model R² on test set: {test_r2_gb:.2f}")

print(f"Gradient Boosting model MSE on training set: {train_mse_gb:.2f}")
print(f"Gradient Boosting model MSE on test set: {test_mse_gb:.2f}")

print(f"Gradient Boosting model RMSE on training set: {train_rmse_gb:.2f}")
print(f"Gradient Boosting model RMSE on test set: {test_rmse_gb:.2f}")

print(f"Gradient Boosting model MAE on training set: {train_mae_gb:.2f}")
print(f"Gradient Boosting model MAE on test set: {test_mae_gb:.2f}")

# Feature importance
importances_gb = best_gb_model.feature_importances_
important_features_gb = pd.Series(importances_gb, index=X_train.columns).sort_values(ascending=False)

# Print the top 10 most important features
print("Top 10 Most Important Features for Gradient Boosting:")
print(important_features_gb.head(10))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Parameters for Gradient Boosting: {'subsample': 0.9, 'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 5, 'learning_rate': 0.05}
Gradient Boosting model R² on training set: 0.76
Gradient Boosting model R² on test set: 0.68
Gradient Boosting model MSE on training set: 639899948.09
Gradient Boosting model MSE on test set: 889046926.73
Gradient Boosting model RMSE on training set: 25296.24
Gradient Boosting model RMSE on test set: 29816.89
Gradient Boosting model MAE on training set: 18834.55
Gradient Boosting model MAE on test set: 22090.36
Top 10 Most Important Features for Gradient Boosting:
Country                   0.610813
YearsCodePro              0.119300
ProfessionalTech          0.061277
WorkExp                   0.052250
OpSysProfessional use     0.040870
DevType                   0.029603
DatabaseHaveWorkedWith    0.016246
WebframeHaveWorkedWith    0.016065
Industry                

In [221]:
used_features = X_train.columns.tolist()
print("Features used in the model:")
print(used_features)

Features used in the model:
['Country', 'YearsCodePro', 'WorkExp', 'OpSysProfessional use', 'ProfessionalTech', 'DevType', 'Industry', 'EdLevel', 'Age', 'LanguageHaveWorkedWith', 'RemoteWork', 'Employment', 'ToolsTechHaveWorkedWith', 'DatabaseHaveWorkedWith', 'WebframeHaveWorkedWith']


In [ ]:
model_filename = r'C:\\Users\\user\\OneDrive - Sri Lanka Institute of Information Technology\\Desktop\\FDM min Project\\mlapp\\best__model.pkl'
features_filename = r'C:\\Users\\user\\OneDrive - Sri Lanka Institute of Information Technology\\Desktop\\FDM min Project\\mlapp\\used_features.pkl'

# Save the model
with open(model_filename, 'wb') as model_file:
    pickle.dump(best_gb_model, model_file)

# Save the features used
with open(features_filename, 'wb') as features_file:
    pickle.dump(used_features, features_file)

print(f"Model and features saved to: {model_filename} and {features_filename}")